In [ ]:
# gcn_builtin_model.py
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn.models import GCN

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1️⃣ Tải dataset Cora
dataset = Planetoid(root="./data", name="Cora", transform=NormalizeFeatures())
data = dataset[0].to(DEVICE)

print(f"Dataset: {dataset.name}")
print(f"Số node: {data.num_nodes}, số cạnh: {data.num_edges // 2}, số feature: {dataset.num_features}, số lớp: {dataset.num_classes}")

# 2️⃣ Khởi tạo mô hình GCN có sẵn
# torch_geometric.nn.models.GCN(in_channels, hidden_channels, out_channels, ...)
model = GCN(
    in_channels=dataset.num_features,   # số feature mỗi node
    hidden_channels=16,                 # số ẩn trong layer giữa
    out_channels=dataset.num_classes,   # số lớp cần phân loại
    num_layers=2,                       # mặc định 2 layer
    dropout=0.5
).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# 3️⃣ Hàm train/test
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)               # forward
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=1)
    accs = []
    for mask_name in ["train_mask", "val_mask", "test_mask"]:
        mask = getattr(data, mask_name)
        acc = (pred[mask] == data.y[mask]).float().mean().item()
        accs.append(acc)
    return accs  # [train, val, test]

# 4️⃣ Train nhanh 200 epoch
for epoch in range(1, 201):
    loss = train()
    if epoch % 20 == 0 or epoch == 1:
        accs = test()
        print(f"[{epoch:03d}] loss={loss:.4f} | train={accs[0]:.3f} val={accs[1]:.3f} test={accs[2]:.3f}")
